In [436]:
import pandas as pd
import numpy_financial as npf
import numpy as np

In [437]:
df = pd.read_excel('123.xlsx', sheet_name='ДЧП (2)')
df.head()

,Unnamed: 0,Дата начала,Годовая ставка,Месяцев,Сумма кредита,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,2020-08-01 00:00:00,0.24,120,375000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Месяц,Платеж,Проценты,ДЧП,Долг,NaN,Месяц,Остаток долга,Погашение кредита,Начисленные проценты,Начисленные проценты (пеня),Погашение основного долга,Погашение ДЧП,Остаток
2,2020-08-01 00:00:00,-1507762.36,-739726.03,-2237.64,374229726.03,NaN,2020-08-10 00:00:00,NaN,1510000,739726.03,NaN,768036.33,2237.64,NaN
3,2020-09-01 00:00:00,-8267986.895281,-7484594.5206,-1588422.64,371857911.01532,NaN,2020-09-10 00:00:00,NaN,10000000,7628180.3,NaN,783397.06,1588422.64,NaN
4,2020-10-01 00:00:00,-8232819.706884,-7437158.220306,-1342379.78,369719869.748742,NaN,2020-10-10 00:00:00,NaN,10000000,7857767.1,788.12,799065,1342379.78,NaN


In [438]:
# Only keep Unnamed: 6 and Unnamed: 8
df = df[['Unnamed: 6', 'Unnamed: 8']]
# Drop first 2 rows
df = df.drop([0, 1])
# Rename Unnamed: 6 to 'date'
df = df.rename(columns={'Unnamed: 6': 'date'})
# Rename Unnamed: 8 to 'factual_payment'
df = df.rename(columns={'Unnamed: 8': 'factual_payment'})
df.head()

,date,factual_payment
2,2020-08-10 00:00:00,1510000
3,2020-09-10 00:00:00,10000000
4,2020-10-10 00:00:00,10000000
5,2020-11-10 00:00:00,5370000
6,2020-12-10 00:00:00,9000000


In [439]:
df.tail()

,date,factual_payment
118,2030-04-10 00:00:00,NaN
119,2030-05-10 00:00:00,NaN
120,2030-06-10 00:00:00,NaN
121,2030-07-10 00:00:00,NaN
122,NaN,372305660.54


In [440]:
# drop missing rows with nan date
df = df.dropna(subset=['date'])

In [441]:
df_copy = df.copy()

In [442]:
# Convert factual_payment to float
df['factual_payment'] = df['factual_payment'].astype(float)

In [443]:
df.head()

,date,factual_payment
2,2020-08-10 00:00:00,1510000.0
3,2020-09-10 00:00:00,10000000.0
4,2020-10-10 00:00:00,10000000.0
5,2020-11-10 00:00:00,5370000.0
6,2020-12-10 00:00:00,9000000.0


In [444]:
# Convert date column to datetime format
df['date'] = pd.to_datetime(df['date'])
# Sum up factual_payment of the rows with the same dates
df = df.groupby(df['date'].dt.date)['factual_payment'].sum().reset_index()
# Sort the dataframe by date
df = df.sort_values(by='date')
df.head(10)

,date,factual_payment
0,2020-08-10,1510000.0
1,2020-09-10,10000000.0
2,2020-10-10,10000000.0
3,2020-11-10,5370000.0
4,2020-12-10,9000000.0
5,2021-01-10,9005000.0
6,2021-02-10,10006000.0
7,2021-03-10,9000000.0
8,2021-04-10,9000000.0
9,2021-05-10,9000000.0


In [445]:
# Function to update graph on a daily basis
def update_graph_daily(balance, annual_interest, df):
    flag = True
    flag2 = True
    previous_date = '2020-07-10 00:00:00'
    previous_date = pd.to_datetime(previous_date)
    # Get the deadline date, which the last row of the date column
    deadline_date = pd.to_datetime(df['date'].iloc[-1])
    daily_interest = (annual_interest / 365)
    for index, row in df.iterrows():
        # Get the date and payment with index row
        current_date = pd.to_datetime(row['date'])
        payment = row['factual_payment']
        days_past = (current_date - previous_date).days
        if payment == 0:
            payment = average_payment * days_past
            flag = False
        # calculate the number of days left until the deadline
        days_left = (deadline_date - current_date).days
        days_past = (current_date - previous_date).days
        interest_amount = balance * daily_interest * days_past
        payment -= interest_amount
        balance -= payment
        if flag or flag2:
            if not flag:
                flag2 = False
            average_payment = (balance * daily_interest) / (1 - (1 + daily_interest)**(-days_left))
        df.loc[index, 'average_payment'] = int(average_payment)
        df.loc[index, 'interest_amount'] = int(interest_amount)
        df.loc[index, 'balance'] = int(balance)
        df.loc[index, 'days_left'] = int(days_left)
        df.loc[index, 'principle'] = int(payment)
        df.loc[index, 'days_past'] = int(days_past)
        previous_date = current_date
    df.to_csv('nuevo.csv', index=True)

In [446]:
balance = 375000000.0
interest = 0.24
months = 120
df_copy = df.copy()
update_graph_daily(balance, interest, df_copy)

In [447]:
df_copy.head()

,date,factual_payment,average_payment,interest_amount,balance,days_left,principle,days_past
0,2020-08-10,1510000.0,276163.0,7643835.0,381133835.0,3621.0,-6133835.0,31.0
1,2020-09-10,10000000.0,275124.0,7768865.0,378902700.0,3590.0,2231134.0,31.0
2,2020-10-10,10000000.0,273859.0,7474245.0,376376945.0,3560.0,2525754.0,30.0
3,2020-11-10,5370000.0,276140.0,7671902.0,378678848.0,3529.0,-2301902.0,31.0
4,2020-12-10,9000000.0,275623.0,7469829.0,377148677.0,3499.0,1530170.0,30.0


In [448]:
df_russian = df_copy.copy()
df_russian.rename(columns={'date':'Дата'}, inplace=True)
df_russian.rename(columns={'factual_payment':'Погашение кредита'}, inplace=True)
df_russian.rename(columns={'interest_amount':'Погашение процентов'}, inplace=True)
df_russian.rename(columns={'principle':'Погашение основного долга'}, inplace=True)
df_russian.rename(columns={'balance':'Остаток долга'}, inplace=True)
df_russian.rename(columns={'average_payment':'Дневной платеж'}, inplace=True)
df_russian.rename(columns={'days_left':'Осталось дней'}, inplace=True)
df_russian.rename(columns={'days_past':'Прошло дней'}, inplace=True)
# Change the order of columns
df_russian = df_russian[['Дата', 'Погашение кредита', 'Погашение процентов', 'Погашение основного долга', 'Остаток долга', 'Дневной платеж', 'Осталось дней', 'Прошло дней']]
df_russian.to_csv('russian.csv', index=False)
df_russian.to_excel('russian.xlsx', index=False)
df_russian.head()

,Дата,Погашение кредита,Погашение процентов,Погашение основного долга,Остаток долга,Дневной платеж,Осталось дней,Прошло дней
0,2020-08-10,1510000.0,7643835.0,-6133835.0,381133835.0,276163.0,3621.0,31.0
1,2020-09-10,10000000.0,7768865.0,2231134.0,378902700.0,275124.0,3590.0,31.0
2,2020-10-10,10000000.0,7474245.0,2525754.0,376376945.0,273859.0,3560.0,30.0
3,2020-11-10,5370000.0,7671902.0,-2301902.0,378678848.0,276140.0,3529.0,31.0
4,2020-12-10,9000000.0,7469829.0,1530170.0,377148677.0,275623.0,3499.0,30.0
